<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание" data-toc-modified-id="Описание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li></ul></div>

# Медицинский дашборд "Показатели эффективности работы онкологической службы"

## Описание

Использованные данные: Состояние онкологической помощи населению России в 2021 году под редакцией А.Д. КАПРИНА, В.В. СТАРИНСКОГО, А.О. ШАХЗАДОВОЙ. Таблицы 22, 24-53.

Цель данного дашборда: наглядно отобразить данные, которые характеризуют состояние онкологической помощи в России в 2021 году.

Целевая аудитория: Врачи и организаторы здравоохранения, а так же все заинтересованные темой

В этой тетрадке содержится предобработка данных для дашборда

Визуализация показателей эффективности онкологической помощи. 2021 год
Дашборд выполнил Бакакин П.А.
ГАУДПО «Уральский институт управления здравоохранением им. А.Б. Блохина» МИАЦ СО.
Telegram: @Pavel-bakakin
Email: pasha-bakakin@yandex.ru

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import regex as re
import geopandas as gpd
import geojson
import shapely

Определяем функцию для загрузки таблиц

In [2]:
def load_data(directory ='2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/'):
    """
    Функция для загрузки данных из таблиц, лежащих в одной дирректории
    Input: Название директории
    Output: Лист с базами в формате pd.DataFrame
    """
    # Создаем пустой список для будующих баз
    datas = list()
    # Задаем название столбцов для таблиц 22 и других
    columns_22 = ['region', 'abs_count', 'to_died_zno', 'to_just_diagnosed', 'abs_after_death',
                           'just_diagnosed_after_death', 'abs_autopsy_after_death', 'just_diagnosed_autopsy_after_death',
                          'percent_all_after_death']
    columns_other = ['region','take_first', 'percent_active', 'at_end_of_year_abs', 'at_end_of_year_intens', 'five_year_abs',
                     'five_year_rate', 'rate_cum_contingent', 'letality', 'registred_zno', 'verifed_morph', 
                     'first_st', 'second_st', 'third_st', 'fourth_st', 'unknown_st', 'first_year_letality']
    j = 0
    # Загружаем таблицы лежащие в одной директории
    while j < len(os.listdir(directory)):
        i = os.listdir(directory)[j]
        # Сначала определяем header и данные таблицы загружая небольшое кол-во строк
        _tmp = pd.read_excel(directory+i, sheet_name=0, nrows=10)
        header = _tmp[_tmp.iloc[:,0] == 'РОССИЯ'].index[0] + 1
        meta = _tmp.columns[0]
        # Загружаем непосредственно таблицу. Таблицу 22, 24, 52 и 53 приходится обрабатывать особым образом
        _tmp = pd.read_excel(directory+i, sheet_name=0, header=header)
        if i.split('_')[2].replace('0','', 1) == '22':
            # Указываем название колонок сохраняем информацию о таблице
            _tmp.columns = columns_22
            _tmp.attrs['table_name'] = meta
            _tmp.dropna(inplace=True)
            # Проверяем кол-во строк, если меньше 92 - значит какая-то ошибка
            if _tmp.shape[0] < 92:
                raise ValueError("Несовпадение числа строк")
            datas.append(_tmp)
            j += 1
            continue
            
        
        # Другие таблицы обрабатываем сходным образом
        if i.split('_')[2].replace('0','', 1) == '24':
            j += 1
            i = os.listdir(directory)[j]
            _tmp_1 = pd.read_excel(directory+i, sheet_name=0, nrows=10)
            header = _tmp_1[_tmp_1.iloc[:,0] == 'РОССИЯ'].index[0] + 1
            _tmp_1 = pd.read_excel(directory+i, sheet_name=0, header=header)
            # Таблицу 24 которая содержится в двух файлах объединяем в одну
            _tmp = _tmp_1.merge(_tmp, how='outer', on = 'РОССИЯ')
            _tmp.columns = columns_other
            _tmp.attrs['table_name'] = meta
            _tmp.dropna(inplace=True)
            if _tmp.shape[0] < 92:
                raise ValueError("Несовпадение числа строк")
            datas.append(_tmp)
            j += 1
            continue
        
        # У таблицы 52 и 53 нет второго листа, обрабатываем особо
        if i.split('_')[2].replace('0','', 1) in ['52', '53']:
            _tmp.columns = columns_other[:9]
            _tmp.attrs['table_name'] = meta
            _tmp.dropna(inplace=True)
            if _tmp.shape[0] < 92:
                raise ValueError("Несовпадение числа строк")
            datas.append(_tmp)
            j += 1
            continue
       
        # Для всех остальных таблиц добавляем второй лист
        _tmp_1 = pd.read_excel(directory+i, sheet_name=1, nrows=10)
        print(directory+i)
        header = _tmp_1[_tmp_1.iloc[:,0] == 'РОССИЯ'].index[0] + 1
        _tmp_1 = pd.read_excel(directory+i, sheet_name=1, header=header)
        # Обхединяем второй лист с первым
        _tmp = _tmp.merge(_tmp_1, how='outer', on = 'РОССИЯ')
        _tmp.columns = columns_other
        _tmp.attrs['table_name'] = meta
        _tmp.dropna(subset='take_first', inplace=True)
        if _tmp.shape[0] < 92:
            raise ValueError("Несовпадение числа строк")
        # Добавляем к списку с таблицами
        datas.append(_tmp)
        j += 1
        
    return datas

С помощью функции загружаем таблицы

In [3]:
datas = load_data()

2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_025_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_026_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_027_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_028_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_029_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_030_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_031_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_033_Состояние_онко_помощи_в_РФ.xlsx
2021_Состояние_онко_помощи_в_РФ_Таблицы_022_и_024-053/2021_Таблица_034_Со

Посмотрим какие таблицы загрузились

In [6]:
for i in range(1,31):
    print(datas[i].attrs['table_name'].split('\n')[-1].split('           ')[0])

Злокачественные новообразования - всего (С00-96)
У детей в возрасте 0-14 лет (С00-96)
У детей в возрасте 0-17 лет (С00-96)
Губа (С00)
Полость рта (C01-09)
Глотка (С10-13)
Пищевод (С15)
Желудок (С16)
Ободочная кишка (С18)
Прямая кишка, ректосиг. соединение, анус (С19-21)
Печень и внутрипеченочные желчные протоки (С22)
Поджелудочная железа (С25)
Гортань (С32)
Трахея, бронхи, легкое (С33,34)
Кости и суставные хрящи (С40,41)
Меланома кожи (С43)
Кожа (кроме меланомы) (С44)
Соединительная и другие мягкие ткани (С47,49)
Молочная железа (С50)
Шейка матки (С53)
Тело матки (С54)
Яичник (С56)
Предстательная железа (С61)
Почка (С64)
Мочевой пузырь (С67)
Щитовидная железа (С73)
Злокачественные лимфомы (С81-86;88,90,96)
Лейкемии (С91-95)
У лиц старше 65 лет (С00-96)
У сельских жителей (С00-96)


С помощью функции выявим строки с суммарной информацией по федеральному округу. Название федерального окргуа выведем в отдельную колонку. Саму строки выбросим

In [5]:
def new_col(row):
    """
    Функция для перевода строки с федеральным округом в колонку
    Input: строка таблицы
    Output: строка с названием округа. Так же сохраняет в глобальной переменной лист
            с индексами строк с федеральными округами: indexes
    """
    global reg
    global indexes
    if row[0].find('ФО') > -1:
        reg = row[0]
        indexes.append(row.name)
    return reg
    

Применяем функцию к каждой таблице, избавляясь от строк с федеральными округами

In [8]:
for i in datas:
    reg = ''
    indexes =[]
    i['district'] = i.apply(lambda x: new_col(x), axis=1)
    i.drop(indexes, axis=0, inplace=True)

Создаем колонки с кодом МКБ и локализацией (или группой) для каждой таблице на основе наименования таблицы. Используем для этого регулярные выражения

In [12]:
for i in range(1,31):
    datas[i]['MKB'] = re.findall(r"\(С.+\)|\(C.+\)", datas[i].attrs['table_name'].split('\n')[-1].split('           ')[0])[0]\
                        .replace('(','').replace(')','')
    datas[i]['type'] = datas[i].attrs['table_name'].split('\n')[-1].split('           ')[0]\
                       .replace(f" ({datas[i].loc[1,'MKB']})", '')
    

Объединяем в одну таблицу таблицы с группами пациентов

In [14]:
df_group = pd.concat([datas[1],datas[2],datas[3], datas[29], datas[30]])

In [15]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 1 to 92
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   region                 425 non-null    object 
 1   take_first             425 non-null    int64  
 2   percent_active         425 non-null    float64
 3   at_end_of_year_abs     425 non-null    int64  
 4   at_end_of_year_intens  425 non-null    float64
 5   five_year_abs          425 non-null    int64  
 6   five_year_rate         425 non-null    float64
 7   rate_cum_contingent    425 non-null    float64
 8   letality               425 non-null    float64
 9   registred_zno          253 non-null    float64
 10  verifed_morph          253 non-null    float64
 11  first_st               253 non-null    float64
 12  second_st              253 non-null    float64
 13  third_st               253 non-null    float64
 14  fourth_st              253 non-null    float64
 15  unknown

Обхединяем в одну таблицу таблицы с локализациями ЗНО

In [16]:
df_mkb = pd.concat([datas[x] for x in range(4,29)])

In [17]:
df_mkb.head()

,region,take_first,percent_active,at_end_of_year_abs,at_end_of_year_intens,five_year_abs,five_year_rate,rate_cum_contingent,letality,registred_zno,verifed_morph,first_st,second_st,third_st,fourth_st,unknown_st,first_year_letality,district,MKB,type
1,Белгородская область,17.0,47.1,357.0,23.1,243.0,68.1,21.0,0.6,23.0,100.0,47.8,30.4,8.7,8.7,4.3,15.8,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
2,Брянская область,17.0,47.1,300.0,25.3,220.0,73.3,17.6,0.7,20.0,100.0,35.0,40.0,20.0,5.0,0.0,9.1,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
3,Владимирская область,16.0,12.5,275.0,20.4,201.0,73.1,17.2,1.8,18.0,100.0,22.2,55.6,16.7,5.6,0.0,7.1,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
4,Воронежская область,44.0,34.1,767.0,33.1,588.0,76.7,17.4,0.9,47.0,100.0,57.4,21.3,17.0,4.3,0.0,2.3,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
5,Ивановская область,19.0,47.4,189.0,19.1,132.0,69.8,9.9,0.0,20.0,100.0,50.0,45.0,0.0,5.0,0.0,0.0,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа


In [18]:
df_mkb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2125 entries, 1 to 92
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   region                 2125 non-null   object 
 1   take_first             2125 non-null   float64
 2   percent_active         2125 non-null   float64
 3   at_end_of_year_abs     2125 non-null   float64
 4   at_end_of_year_intens  2125 non-null   float64
 5   five_year_abs          2125 non-null   float64
 6   five_year_rate         2125 non-null   float64
 7   rate_cum_contingent    2125 non-null   float64
 8   letality               2125 non-null   float64
 9   registred_zno          2121 non-null   float64
 10  verifed_morph          2121 non-null   float64
 11  first_st               2121 non-null   object 
 12  second_st              2121 non-null   object 
 13  third_st               2121 non-null   object 
 14  fourth_st              2121 non-null   object 
 15  unknow

In [19]:
df_group

,region,take_first,percent_active,at_end_of_year_abs,at_end_of_year_intens,five_year_abs,five_year_rate,rate_cum_contingent,letality,registred_zno,verifed_morph,first_st,second_st,third_st,fourth_st,unknown_st,first_year_letality,district,MKB,type
1,Белгородская область,6134,27.4,44114,2854.9,24836,56.3,7.2,5.1,6925.0,94.5,36.2,26.2,15.3,18.6,3.7,18.7,ЦЕНТРАЛЬНЫЙ ФО,С00-96,Злокачественные новообразования - всего
2,Брянская область,5137,20.3,38258,3221.5,21775,56.9,7.4,6.0,5702.0,97.1,31.3,18.6,17.0,28.6,4.6,26.4,ЦЕНТРАЛЬНЫЙ ФО,С00-96,Злокачественные новообразования - всего
3,Владимирская область,4986,10.6,44686,3309.4,26554,59.4,9.0,4.1,5503.0,96.8,30.4,23.7,18.0,23.5,4.5,20.5,ЦЕНТРАЛЬНЫЙ ФО,С00-96,Злокачественные новообразования - всего
4,Воронежская область,8170,29.2,71536,3090.2,41212,57.6,8.8,5.1,9021.0,94.2,38.4,26.6,15.0,18.5,1.5,18.8,ЦЕНТРАЛЬНЫЙ ФО,С00-96,Злокачественные новообразования - всего
5,Ивановская область,4129,28.9,31860,3211.4,18064,56.7,7.7,5.1,4587.0,98.0,33.6,21.0,14.6,23.8,7.1,24.0,ЦЕНТРАЛЬНЫЙ ФО,С00-96,Злокачественные новообразования - всего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Забайкальский край,1039,17.7,8402,6727.0,5048,60.1,8.1,7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ДАЛЬНЕВОСТОЧНЫЙ ФО,С00-96,У сельских жителей
89,Чукотский авт.округ,29,20.7,124,3903.7,64,51.6,4.3,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ДАЛЬНЕВОСТОЧНЫЙ ФО,С00-96,У сельских жителей
90,Республика Бурятия,1314,27.8,8546,7484.1,4574,53.5,6.5,8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ДАЛЬНЕВОСТОЧНЫЙ ФО,С00-96,У сельских жителей
91,Республика Саха (Якутия),599,29.2,3072,3466.2,1539,50.1,5.1,10.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ДАЛЬНЕВОСТОЧНЫЙ ФО,С00-96,У сельских жителей


In [20]:
datas[0]

,region,abs_count,to_died_zno,to_just_diagnosed,abs_after_death,just_diagnosed_after_death,abs_autopsy_after_death,just_diagnosed_autopsy_after_death,percent_all_after_death,district
1,Белгородская область,112,4.6,1.8,112,1.8,112,1.8,100.0,ЦЕНТРАЛЬНЫЙ ФО
2,Брянская область,236,8.8,4.6,234,4.6,234,4.6,100.0,ЦЕНТРАЛЬНЫЙ ФО
3,Владимирская область,598,23.7,12.0,589,11.8,585,11.7,99.3,ЦЕНТРАЛЬНЫЙ ФО
4,Воронежская область,172,4.3,2.1,172,2.1,172,2.1,100.0,ЦЕНТРАЛЬНЫЙ ФО
5,Ивановская область,199,10.5,4.8,199,4.8,199,4.8,100.0,ЦЕНТРАЛЬНЫЙ ФО
...,...,...,...,...,...,...,...,...,...,...
88,Забайкальский край,32,1.8,1.0,32,1.0,32,1.0,100.0,ДАЛЬНЕВОСТОЧНЫЙ ФО
89,Чукотский авт.округ,11,17.7,10.3,10,9.3,10,9.3,100.0,ДАЛЬНЕВОСТОЧНЫЙ ФО
90,Республика Бурятия,75,4.4,2.4,75,2.4,75,2.4,100.0,ДАЛЬНЕВОСТОЧНЫЙ ФО
91,Республика Саха (Якутия),203,16.3,9.5,203,9.5,203,9.5,100.0,ДАЛЬНЕВОСТОЧНЫЙ ФО


In [26]:
df_mkb

,region,take_first,percent_active,at_end_of_year_abs,at_end_of_year_intens,five_year_abs,five_year_rate,rate_cum_contingent,letality,registred_zno,verifed_morph,first_st,second_st,third_st,fourth_st,unknown_st,first_year_letality,district,MKB,type
1,Белгородская область,17.0,47.1,357.0,23.1,243.0,68.1,21.0,0.6,23.0,100.0,47.8,30.4,8.7,8.7,4.3,15.8,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
2,Брянская область,17.0,47.1,300.0,25.3,220.0,73.3,17.6,0.7,20.0,100.0,35.0,40.0,20.0,5.0,0.0,9.1,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
3,Владимирская область,16.0,12.5,275.0,20.4,201.0,73.1,17.2,1.8,18.0,100.0,22.2,55.6,16.7,5.6,0.0,7.1,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
4,Воронежская область,44.0,34.1,767.0,33.1,588.0,76.7,17.4,0.9,47.0,100.0,57.4,21.3,17.0,4.3,0.0,2.3,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
5,Ивановская область,19.0,47.4,189.0,19.1,132.0,69.8,9.9,0.0,20.0,100.0,50.0,45.0,0.0,5.0,0.0,0.0,ЦЕНТРАЛЬНЫЙ ФО,С00,Губа
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Забайкальский край,52.0,1.9,469.0,44.4,294.0,62.7,9.0,10.7,58.0,100.0,-,-,-,-,-,27.3,ДАЛЬНЕВОСТОЧНЫЙ ФО,С91-95,Лейкемии
89,Чукотский авт.округ,1.0,100.0,7.0,14.0,2.0,28.6,7.0,0.0,1.0,100.0,-,-,-,-,-,0.0,ДАЛЬНЕВОСТОЧНЫЙ ФО,С91-95,Лейкемии
90,Республика Бурятия,29.0,6.9,311.0,31.6,184.0,59.2,10.7,7.2,32.0,100.0,-,-,-,-,-,30.8,ДАЛЬНЕВОСТОЧНЫЙ ФО,С91-95,Лейкемии
91,Республика Саха (Якутия),52.0,11.5,379.0,38.8,238.0,62.8,7.3,6.0,53.0,100.0,-,-,-,-,-,25.0,ДАЛЬНЕВОСТОЧНЫЙ ФО,С91-95,Лейкемии


Загружаем данные с координатами округов полученные из wiki DataLens

In [29]:
geo_yand = pd.read_csv('ru_regions.csv', sep=';')

In [30]:
geo_yand

,name,type,id,region,coords_type,coords
0,Ростовская,Область,64,Ростовская область,ru_regions_poly,"[[[50.21447689972731,41.3502545996028],[50.199..."
1,Рязанская,Область,65,Рязанская область,ru_regions_poly,"[[[55.36424399994077,40.210444700223796],[55.3..."
2,Сахалинская,Область,69,Сахалинская область,ru_regions_poly,"[[[46.4051481123399,142.82388437012776],[46.39..."
3,Самарская,Область,66,Самарская область,ru_regions_poly,"[[[54.67642609980043,51.38997859973747],[54.66..."
4,Саратовская,Область,68,Саратовская область,ru_regions_poly,"[[[52.81440689976361,48.42863240015605],[52.76..."
...,...,...,...,...,...,...
165,Кемеровская,Область,18,Кемеровская область,ru_regions_points,"[54.8259915192,87.1985380848]"
166,Еврейская,Автономная Область,11,Еврейская автономная область,ru_regions_points,"[48.5274438727,132.258418362]"
167,Пермский,Край,39,Пермский край,ru_regions_points,"[59.2009037329,56.2278211786]"
168,Московская,Область,29,Московская область,ru_regions_points,"[55.2887137973,38.9413900249]"


In [31]:
geo_yand = geo_yand[['region','coords']].rename({'coords':'coordinates'}, axis=1)

Отбираем координаты полигонов с округами: у них строка с координатами длинее чем у точек с центрами округов

In [32]:
geo_yand = geo_yand[geo_yand.coordinates.apply(lambda x: len(x)>40)]

In [34]:
geo_yand.reset_index(drop=True, inplace=True)

Определяем функцию для получения координат из базы в наши таблицы

In [35]:
def get_coordinates(row):
    """
    Функция для добавления колонки с координатами региона в таблицу
    Input: строка таблицы
    Output: строка с координатами
    """
    # Список для удаления ненужных постфиксов
    list_del = ['край', 'область', 'обл.(б/а.о)', 'авт. обл.', 'а.о.', 'респ.', 'республика', 'обл.'
                , 'авт.округ', 'г.', 'город ']
    # из нашей таблицы получаем название региона и удаляем из него все лишнее
    name = row.region.lower()
    for i in list_del:
        name = name.replace(i, '')
    name = name.replace('-', ' ').replace('—', ' ').replace('c', 'с').strip().replace('(', '').replace(')', '')
    # Ненецкий обработаем отдельно, чтобы не совпал с Ямало-Ненецким
    if name == 'ненецкий':
        coord_region = str(geo_yand.loc[geo_yand[geo_yand.region == 'Ненецкий автономный округ'].index[0], 'coordinates'])
        return coord_region
    # Определяем индекс строки содержащей координаты
    ind = geo_yand[geo_yand.region.str.lower().str.replace('-', ' ').str.replace('c', 'с')
                       .str.replace('(', '').str.replace(')', '')
                       .str.replace('—', ' ').str.strip().str.contains(r'\b'+name, regex=True)].index[0]
    # Если индекс найден - записываем в переменную
    if ind:
        coord_region = str(geo_yand.loc[ind, 'coordinates'])
    # Если не найден - обрезаем название региона до трех букв и пытаемся снова
    else:
        ind = geo_yand[geo_yand.region.str.lower().str.replace('-', ' ', regex=False).str.replace('c', 'с', regex=False)
                       .str.replace('—', ' ', regex=False).str.strip().str.contains(r'\b'+name[:3], regex=True)].index[0]
        coord_region = str(geo_yand.loc[ind, 'coordinates'])
    return coord_region

Добавляем колонки с координатами в три объединенные таблицы

In [37]:
datas[0]['coord_region'] = datas[0].apply(lambda x: get_coordinates(x), axis=1)

In [38]:
df_mkb['coord_region'] = df_mkb.apply(lambda x: get_coordinates(x), axis=1)

In [39]:
df_group['coord_region'] = df_group.apply(lambda x: get_coordinates(x), axis=1)

Сохраняем их

In [40]:
datas[0].to_csv('table_22.csv', index=False)
df_mkb.to_csv('df_mkb.csv', index=False)
df_group.to_csv('df_group.csv', index=False)